# Markr.AI Data Generation

`@ Developer : DopeorNope (Seungyoo Lee), Kyujinpy (Kyunjin Han)`

`@ Updata Date: 2024.03.22`

`@ Description :` 

- AI Hub 데이터와, Public Data를 통해서 Fine-tuning용 데이터를 생성하는 코드입니다.
    
    - 데이터를 생성하기 위해서는 기본 경로에,  '도서자료 요약', '문서요약 텍스트', '018.논문자료 요약 데이터', '048.일반상식 문장 생성 데이터', 'KorQuAD_v1.0_train.json' 데이터를 넣어주시면 됩니다.

    - Chat GPT가 아닌, License에 제약이 없는 방법론으로 기존의 데이터들을 생성해내는 것을 학습 세트로 하여 `Custom LLM을 개발` 할 수 있는 것을 목표로 하여 Code를 공유하기로 하였습니다.

    - 어떤 말뭉치 데이터(corpus)로도 코드르 조금만 수정하면 원하시는 도메인에 맞는 데이터셋을 생성 할 수 있습니다.     

In [ ]:
import os
import pandas as pd
import json
from tqdm import tqdm
from datasets import dataset_dict, load_dataset,Dataset,DatasetDict
import random
import string
import re

In [2]:
default_path = './'
data = os.listdir(default_path)
print(data)

['CoT_collection_ko.json', 'WIKI_QA_Near_dedup.parquet', 'KoAlpaca.parquet', 'ko-platypus.parquet', 'KorQuAD_v1.0_train.json', 'KorQuad2', '.ipynb_checkpoints', 'Make_ko_dataset.ipynb', '도서자료 요약', '문서요약 텍스트', '018.논문자료 요약 데이터', '048.일반상식 문장 생성 데이터', 'Save2', 'instruction_ko_dataset2.ipynb', 'Save', '13']


# 1. Multi question

- `KorQuAD_v1.0_train.json`를 통해서 Data를 생성하는 프로세스입니다.

-  https://korquad.github.io/category/1.0_KOR.html 여기서 데이터를 다운 받으실 수 있습니다.

In [15]:
KorQuAD_v1 = pd.DataFrame(columns=['input', 'instruction', 'output'])

data_path = default_path + 'KorQuAD_v1.0_train.json'
with open(data_path, 'r', encoding='UTF8') as file:
    json_file = json.load(file)
#print(json_file['data'][0])

row = 0
for d in tqdm(json_file['data']):
    paragraph = d['paragraphs'][0]
    context = paragraph['context']
    question = paragraph['qas']
    
    answer_text = ''
    question_text = ''
    number = 1
    for que in question:
        answer_text += str(number) + '. ' + que['answers'][0]['text'] + '\n'
        question_text += str(number) + '. ' + que['question'] + '\n'
        number += 1
    
    instruction = '주어진 정보를 기반으로 질문에 답하세요. 답을 모른다면 답을 지어내지 말고 그냥 모른다고 말하세요.\n\n' + context + '\n\n'
    instruction += '질문:\n' + question_text
    KorQuAD_v1.loc[row] = ['', instruction.strip(), answer_text.strip()]
    row += 1

100%|████████████████████████████████████████████████████████████████████████████| 1420/1420 [00:01<00:00, 1018.58it/s]


In [16]:
print(KorQuAD_v1.iloc[0,1])
print(KorQuAD_v1.iloc[0,2])

주어진 정보를 기반으로 질문에 답하세요. 답을 모른다면 답을 지어내지 말고 그냥 모른다고 말하세요.

1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.

질문:
1. 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
2. 바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?
3. 바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?
4. 1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?
5. 파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?
6. 바그너가 파우스트를 처음으로 읽은 년도는?
7. 바그너가 처음 교향곡 작곡을 한 장소는?
8. 바그너의 1악장의 초연은 어디서 연주되었는가?
1. 교향곡
2

In [17]:
# KorQuAD 데이터셋 저장

KorQuAD_v1.to_parquet(default_path+'Save/KorQuAD_v1_com.parquet', engine = 'pyarrow', index = False)

# 2. 문서 요약 텍스트

-  AI Hub의 `문서요약 텍스트`를 활용하여, 데이터셋을 생성해 내는 파이프라인입니다.

- `문서요약 텍스트`는 다음 링크에서 다운 받으실 수 있습니다.

- https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=97



In [161]:
default = default_path + '문서요약 텍스트/training'
data_folder = os.listdir(default)
df = pd.DataFrame(columns=['input', 'instruction', 'output'])

text_list = []
instruction_list = []
for d in data_folder:
    path = default + '/' + d
    sub_df = pd.DataFrame(columns=['input', 'instruction', 'output'])
    folder2 = os.listdir(path)
    
    for f2 in folder2:
        path = default + '/' + d + '/' + f2
    
        # json 불러오기
        with open(path, 'r', encoding='UTF8') as file:
            json_file = json.load(file)

        #print(json_file['documents'][0]['abstractive'][0])
        documents = json_file['documents']
        for i in tqdm(range(len(documents))):
            title = documents[i]['title']
            text = documents[i]['abstractive'][0]
            sentence_list = documents[i]['text']
            #print(sentence_list)
            #break
            
            number = random.choice([0,1])
            if number == 0:
                instruction_text = '주어진 문장을 적절하게 요약해주세요.\n\n문장: '
                for sentence in sentence_list:
                    #print(sentence)
                    try:
                        if len(sentence) > 1:
                            for sub in sentence:
                                sen = sub['sentence']
                                instruction_text += sen + ' '
                        else:
                            sen = sentence[0]['sentence']
                            instruction_text += sen + ' '
                    except:
                        #print(sentence)
                        pass
                instruction_list.append(instruction_text.strip())
                text_list.append(documents[i]['abstractive'][0])
            else:
                instruction_text = '주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: '
                for sentence in sentence_list:
                    try:
                        if len(sentence) > 1:
                            for sub in sentence:
                                sen = sub['sentence']
                                instruction_text += sen + ' '
                        else:
                            sen = sentence[0]['sentence']
                            instruction_text += sen + ' '
                    except:
                        #print(sentence)
                        pass
                instruction_list.append(instruction_text.strip())
                answer = '제목: ' + title + '\n' + documents[i]['abstractive'][0]
                text_list.append(answer)

    sub_df['input'] = ''
    sub_df['instruction'] = instruction_list
    sub_df['output'] = text_list
    print(sub_df.iloc[0,1])
    print(sub_df.iloc[0,2])
    df = pd.concat([df, sub_df], axis=0)
    text_list = []
    instruction_list = []

100%|███████████████████████████████████████████████████████████████████████| 243983/243983 [00:02<00:00, 93799.94it/s]


주어진 문장을 적절하게 요약해주세요.

문장: ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.
전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주

100%|████████████████████████████████████████████████████████████████████████| 24329/24329 [00:00<00:00, 113836.20it/s]


주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.

문장: 원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고, 노조조합장이 사임한 경우, 노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고 종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고 고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면, 이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.
제목: 부당노동행위구제재심판정취소
원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.


100%|█████████████████████████████████████████████████████████████████████████| 56760/56760 [00:00<00:00, 72887.31it/s]


주어진 문장을 적절하게 요약해주세요.

문장: 이명박 대통령이 어제 30대 그룹 총수를 모아놓고 "시대적 요구는 역시 총수가 앞장서야 한다. 이미 상당한 변화의 조짐이 있다는 것을 고맙게 생각한다. 총수들께서 직접 관심을 가져주시면 빨리 전파돼 긍정적인 평가를 받을 수 있다고 본다"고 말했다. 언뜻 보아 무슨 말인지 불분명하나 이 대통령이 지난 8ㆍ15 연설 후 정몽준 의원, 정몽구 현대차 회장이 각각 2000억원과 5000억원을 기부한 사실과 '공생발전'이란 화두를 연결하면 금방 짐작이 간다. 다른 그룹 총수들도 좀 나서라고 은근히 떠민 것이다. 이 대통령은 기부에 대한 후속 선언이 나오지 않은 탓인지 총수들의 사회공헌 방안에 불만을 표시했다는 후문이다. 최근 미국 프랑스 벨기에 등에서 부유세가 거론되고 독일조차 2년간 한시적으로 5%의 자산세를 거둬 약 155조원을 마련하자는 논의가 있었다. 이런 흐름에 한국만 동떨어져 있기는 어려운 게 글로벌 시대의 특징이다. 항간에는 이번 회동 후 삼성을 비롯해 몇몇 그룹이 노블레스 오블리주 방안을 준비하고 있다는 말이 나도는데 대통령의 강요나 포퓰리즘에 의한 압박보다 자발적 문화로 만들어가야 효과가 큰 법이다. 그런 면에서 재계에 적절한 방안 마련을 맡기고 정치권이나 여론은 너무 압박하지 말고 시간을 줘야 한다. 국가채무 문제로 글로벌 경기 침체 우려가 큰 상황에서 기업들은 '생존'에 큰 부담을 느끼고 있기 때문이다. 이날 전경련에 따르면 30대 그룹은 올해 고용 12만4000명, 투자 114조원 등 '선물'을 준비했다. 세계적인 더블딥이 우려되는 상황에서 공격경영이 어렵겠지만 연초 한 번 발표한 내용을 약간 수정해 내놓은 전경련의 행태는 답답하다. 설립 50주년이 됐으면 좀 더 창의적이고 유연하게 바뀔 때도 됐다. 허창수 전경련 회장은 "대기업ㆍ중소기업이 서로 공생하고 발전할 수 있도록 노력하겠다. 기업이 사회적 책임을 다하겠다"는 원론적인 발언에 그쳐 전경련 특유의 무미건조함을 드러냈다. 한편 이건희 삼성전자 회장

In [162]:
df.reset_index(drop=True, inplace=True)

In [163]:
df['input'] = ''

In [164]:
df

,input,instruction,output
0,,주어진 문장을 적절하게 요약해주세요.\n\n문장: ha당 조사료 400만원…작물별 ...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,,"주어진 문장을 적절하게 요약해주세요.\n\n문장: 8억 투입, 고소천사벽화·자산마을...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 전남드래곤...",제목: “새해 정기 받고 올해는 반드시 일내자!”\n전남드래곤즈 임직원과 선수단이 ...
3,,"주어진 문장을 적절하게 요약해주세요.\n\n문장: 11~24일, 매실·감·참다래 등...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,,주어진 문장을 적절하게 요약해주세요.\n\n문장: 홍콩 크루즈선사‘아쿠아리우스’ 4...,올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...
...,...,...,...
325067,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 판문점 북...","제목: [사설] 北, 남북대화도 주저하지 말라\n북미 관계는 정상궤도에 올랐으나 남..."
325068,,"주어진 문장을 적절하게 요약해주세요.\n\n문장: 참의원 선거 앞둔 아베, 노림수 ...",아베 일본 정부는 일본 강제징용 피해자에게 배상해야 한다는 우리 법원의 판결로 일본...
325069,,"주어진 문장에 적절한 제목을 생성하고, 내용을 요약해주세요.\n\n문장: 미쓰비시중...",제목: [사설] 징용배상 갈등을 수출규제 보복으로 대응하는 日 정부의 졸렬함\n일본...
325070,,주어진 문장을 적절하게 요약해주세요.\n\n문장: 지난달 17 일 학교 비정규직 노...,지난달 17일 학교 비정규직 노동자들이 청와대 앞에서 처우개선을 요구하는 시위를 진...


In [165]:
df.to_parquet(default_path+'Save/summary_com.parquet', engine = 'pyarrow', index = False)

# Text Extracting

- 이 작업은 기존의 AI 허브 데이터셋에서 데이터를 추출해서 corpus 단위로 뭉치는 작업입니다.

- 이를 통해, 말 뭉치를 한데로 모으고, specific한 데이터를 추출하게 됩니다.

- 위 과정을 거쳐야 향후 데이터셋 생성이 가능한 작업이니, 실행하시는 것을 추천 드립니다.

### - 논문자료 요약 데이터 Corpus 저장

In [24]:
######### 단순한 text, doc_id 추출하는 데이터

default = default_path + '/' + data[10] + '/01.데이터/1. Training/라벨링데이터/'
data_folder = os.listdir(default)
df = pd.DataFrame(columns=['text', 'doc_id', 'domain'])

instruction_list = []
text_list = []
doc_id_list = []
doc_id = 0
for d in data_folder:
    path = default + '/' + d
    folder = os.listdir(path)
    
    for f in folder:
        print(f)
        path = default + '/' + d + '/' + f
    
        sub_df = pd.DataFrame(columns=['text', 'doc_id', 'domain'])

        # json 불러오기
        with open(path, 'r', encoding='UTF8') as file:
            json_file = json.load(file)
        
        try:
            documents = json_file[0]['data']
            #print(documents[0])
        except:
            documents = json_file['data']

        for i in tqdm(range(len(documents))):
            text1 = documents[i]['summary_section'][0]['orginal_text']
            text_list.append(text1)
            doc_id_list.append('AIHUB_논문자료요약_'+str(doc_id))
            doc_id += 1
            
        sub_df['domain'] = ''
        sub_df['doc_id'] = doc_id_list
        sub_df['text'] = text_list
        df = pd.concat([df, sub_df], axis=0)
        text_list = []
        instruction_list = []
        doc_id_list = []

논문요약20231006_0.json


100%|███████████████████████████████████████████████████████████████████████| 40000/40000 [00:00<00:00, 1056373.36it/s]


논문요약20231006_1.json


100%|███████████████████████████████████████████████████████████████████████| 40000/40000 [00:00<00:00, 1065132.15it/s]


논문요약20231006_2.json


...

100%|██████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<?, ?it/s]


특허전체_9-Y-7.json


100%|██████████████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:00<?, ?it/s]


특허전체_9-Y-9.json


100%|██████████████████████████████████████████████████████████████████████████| 2500/2500 [00:00<00:00, 467363.17it/s]


In [25]:
df

,text,doc_id,domain
0,‘고전문학교육’을 위해 ‘고전 시대의 문학 교육’의 양상을 살펴보는 일은 그 용어의...,AIHUB_논문자료요약_0,NaN
1,일본의 조선침략 이후 조선의 각 개항장에 진출한 일본인 상업자본가는 소작제 농장경영...,AIHUB_논문자료요약_1,NaN
2,주식시장에서 주식 가격 급등락을 완화하여 가능한 한 가격 형성의 안정성을 유지하기 ...,AIHUB_논문자료요약_2,NaN
3,"아동기암의 성인 생존자에게서 나타나는 신체적, 정신사회적, 교육적 문제는 학령기와 ...",AIHUB_논문자료요약_3,NaN
4,임상에 접한 임상의는 매 순간마다 수많은 의사 결정을 신속히 내려야 한다. 임상의는...,AIHUB_논문자료요약_4,NaN
...,...,...,...
2495,"일반적으로, 선박용 또는 산업용으로 사용되는 디젤엔진에 있어서, 윤활유 계통 설비 ...",AIHUB_논문자료요약_396382,NaN
2496,일반적으로 선박엔진에는 비상사태시 압축공기를 이용하여 엔진으로의 연료 공급을 차단함...,AIHUB_논문자료요약_396383,NaN
2497,위치 기반 서비스란 이동중인 사용자의 위치정보를 다양하고 유용한 정보와 실시간으로 ...,AIHUB_논문자료요약_396384,NaN
2498,"일반적으로 배선용 차단기는 공장, 빌딩 등의 수배전 설비중에서 주로 배전반에 설치되...",AIHUB_논문자료요약_396385,NaN


In [ ]:
df.to_parquet(default_path+'13/논문자료요약.parquet', engine = 'pyarrow', index = False)

### - 일반상식 문장 생성 데이터 Corpus 저장

In [20]:
######### 단순한 text, doc_id 추출하는 데이터

default = default_path + data[11] + '/01-1.정식개방데이터/Training/02.라벨링데이터'
data_folder = os.listdir(default)
df = pd.DataFrame(columns=['text', 'doc_id', 'domain'])

instruction_list = []
text_list = []
doc_list = []
doc_id = 0
for d in data_folder:# TL1
    path = default + '/' + d
    folder = os.listdir(path)
    
    for f in tqdm(folder): # 4개
        path = default + '/' + d + '/' + f

        sub_df = pd.DataFrame(columns=['text', 'doc_id', 'domain'])
        
        try:
            with open(path, 'r', encoding='UTF8') as file:
                json_file = json.load(file)
            
            for documents in json_file:
                text = documents['sentence']
                genSentences = documents['genSentences']
                genChoice = random.choice(genSentences)
                text_list.append(genChoice['label-scenes'].strip())

                instruction_sentence = '임의의 순서대로 나열된 단어들을 보고 적절한 문장을 재구성하세요.\n\n임의의 순서로 나열된 단어: ['
                for concept in documents['concepts']:
                    #print(concept)
                    instruction_sentence += concept['stem'] + ', '
                instruction_sentence = instruction_sentence.strip()[:-1]
                instruction_sentence += ']'
                    
                instruction_list.append(instruction_sentence)
                doc_list.append('AIHUB_일반상식문장생성데이터_'+str(doc_id))
                doc_id += 1
                    
        except Exception as e:
            print(e)
            pass
    
    sub_df['domain'] = ''
    sub_df['text'] = text_list
    sub_df['doc_id'] = doc_list
    df = pd.concat([df, sub_df], axis=0)
    text_list = []
    instruction_list = []
    doc_list = []

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.38s/it]


In [21]:
df

,text,doc_id,domain
0,아버지께 들은 이야기로 인해서 소는 나에게 경계 대상이다.,AIHUB_일반상식문장생성데이터_0,NaN
1,마을은 축제를 구경하기 위해 찾아온 사람들로 붐빈다.,AIHUB_일반상식문장생성데이터_1,NaN
2,그들은 우리의 시각을 자극하는 요소들을 정비했다.,AIHUB_일반상식문장생성데이터_2,NaN
3,나는 옛날 대학 시절부터 오늘날까지 이 책을 읽어 왔다.,AIHUB_일반상식문장생성데이터_3,NaN
4,농부는 농장에서 키우는 말에 고삐를 달아 끌고 갔다.,AIHUB_일반상식문장생성데이터_4,NaN
...,...,...,...
406491,총을 든 그들은 그에게 총격을 가해 왔다.,AIHUB_일반상식문장생성데이터_406491,NaN
406492,그가 직장을 그만두고 나서 힘들게 생활하고 있다.,AIHUB_일반상식문장생성데이터_406492,NaN
406493,그는 그 일을 아무 생각 없이 그대로 따라했다.,AIHUB_일반상식문장생성데이터_406493,NaN
406494,이번 추석에도 멀리 해외에 계신 동포 여러분께 안부 인사를 전합니다.,AIHUB_일반상식문장생성데이터_406494,NaN


In [22]:
df.to_parquet(default_path+'13/일반상식문장생성데이터.parquet', engine = 'pyarrow', index = False)

In [ ]:
# ---> 일반상식문장생성데이터.ipynb로 이동

# 3. Original sentence inference

- AI-Hub의 `도서자료 요약` 데이터를 가지고 Original sentence inference 데이터를 생성하는 파이프 라인입니다.

- `도서자료 요약` 데이터는 다음 링크에서 다운 받을 수 있습니다.


- https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=93 




In [215]:
default = default_path + '도서자료 요약/training'
data_folder = os.listdir(default)
df = pd.DataFrame(columns=['input', 'instruction', 'output'])

text_list = []
instruction_list = []
for d in data_folder:
    path = default + '/' + d
    folder2 = os.listdir(path)
    
    for f2 in folder2:
        path = default + '/' + d + '/' + f2
        folder3 = os.listdir(path)
        
        sub_df = pd.DataFrame(columns=['input', 'instruction', 'output'])
        
        for f3 in tqdm(folder3):
            path = default + '/' + d + '/' + f2 + '/' + f3
    
            # json 불러오기
            with open(path, 'r', encoding='UTF8') as file:
                json_file = json.load(file)

            #print(json_file['documents'][0]['abstractive'][0])
            number = random.choice([0,1])
            documents = json_file['passage']
            documents_split = documents.split('.')
            documents_split = [d0 for d0 in documents_split if d0 != '']
            if len(documents_split) == 1:
                number = 1
            if '%' in documents:
                number = 1
            if number == 0:
                documents = json_file['passage']
                documents_split = documents.split('.')
                documents_split = [d0 for d0 in documents_split if d0 != '']
                doc_text = ''
                for d0 in documents_split[:-1]:
                    doc_text += d0 + '. '
                text = json_file['summary']
                text_list.append(documents_split[-1].strip())

                instruction_text = '주어진 문장 뒤에 자연스럽게 이어질 문장을 생성해주세요.\n\n문장: '
                instruction_text += doc_text.strip()

                instruction_list.append(instruction_text.strip())
                
            elif number == 1:
                documents = json_file['passage']
                text = json_file['summary']
                text_list.append(documents.strip())
                title = json_file['metadata']['doc_name']
                label = json_file['metadata']['kdc_label']

                instruction_text = '주어진 제목과 요약문에 대한 정보를 토대로, 요약되기 전 문장을 유추해서 생성해주세요.\n\n'
                instruction_text += '제목: ' + title + '\n'
                instruction_text += '요약문: ' + text.strip()
                
                instruction_list.append(instruction_text.strip())
                
            else:
                documents = json_file['passage']
                text = json_file['summary']
                text_list.append(text.strip())
                title = json_file['metadata']['doc_name']
                label = json_file['metadata']['kdc_label']

                instruction_text = '주어진 제목과 카테고리에 대한 정보를 토대로, 적합한 문장을 생성해주세요.\n\n'
                instruction_text += '제목: ' + title + '\n'
                instruction_text += '카테고리: ' + '[' + label + ']'
                
                instruction_list.append(instruction_text.strip())

        sub_df['input'] = ''
        sub_df['instruction'] = instruction_list
        sub_df['output'] = text_list
        print(sub_df.iloc[0,1])
        print(sub_df.iloc[0,2])
        df = pd.concat([df, sub_df], axis=0)
        text_list = []
        instruction_list = []

100%|███████████████████████████████████████████████████████████████████████████| 23918/23918 [02:28<00:00, 161.26it/s]


주어진 제목과 요약문에 대한 정보를 토대로, 요약되기 전 문장을 유추해서 생성해주세요.

제목: 수산물 수급 위기관리체계 구축을 위한 기초연구
요약문: 현대 사회에서 발생하는 다양하고 복잡한 위기는 과거와 다르게 국가의 재난으로 이어질 수 있으므로 국가의 신속한 대책 능력이 필요하다.
지금의 국가가 직면하는 위기는 전통사회의 그것과 위기의 규모뿐만아니라 종류와 성질 면에서 사뭇 다르다고 할 수 있다. 즉 현대 사회에서는 매우 다양하고 광범위하게 위기가 발생할 뿐만 아니라 지구온난화에 따른 각종 자연재해로부터 인위적 재난인 사회적 이슈로 인한 위기에 이르기까지 복잡한 양상을 보이고 있다. 따라서 위기의 영향은 한 산업뿐만 아니라 국가의 재난으로 이어지는 경우가 많다. 따라서 국가는 이러한 위기상황이 발생하였을 경우에 그 징후를 빨리 파악하고 대응하지 않으면 안 되는데, 국민의 생명과 재산을 보호하는 것이 바로 국가의 가장 기본적인 의무이기 때문이다.


100%|█████████████████████████████████████████████████████████████████████████████| 6753/6753 [00:14<00:00, 480.16it/s]


주어진 제목과 요약문에 대한 정보를 토대로, 요약되기 전 문장을 유추해서 생성해주세요.

제목: 독일어 형태론
요약문: 언어학은 언어가 갖고 있는 내용면과 표현면의 기호체계를 연구하는 학문이다. 형태론은 언어학의 기초이론 중 한 분야로, 형태소를 중심으로 결합과 교체를 기술하는 것이다. 형태소는 단어의 구성원이 되고, 단어는 문장을 형성한다. 형태소는 문장이나 단어를 분절하여 나타나는 의미를 지닌 최소단위이며, 이를 찾기 위한 분석절차가 필요하다.
제1부. 형태론의 기초
Ⅰ. 형태론의 기본개념들
 형태론은 단어의 내적 구조를 연구하는 언어학의 한 분야이다. Saussure의 정의에 따라 언어를 인간들이 대화를 하기 위한 기호체계로 볼 때, 모든 언어의 기호체계는 감각적으로 인지할 수 있는, 즉 음성적인 연속체로서의 표현면과 의미를 담고 있는 내용면을 가진다. 그리고 이러한 기호체계를 연구하는 학문이 언어학이다. 언어학의 기초이론 분야는 음운론, 형태론, 통사론, 의미론, 화용론 등으로 구성되는데, 음운론(Phonologie)이 음소(Phonem)라는 최소 단위를 중심으로 그 결합과 교체를 기술하는 분야라면 형태론(Morphologie)은 형태소(Morphem)라는 단위를 중심으로 그 결합과 교체를 기술하는 분야라고 할 수 있다. 이 형태소들이 단어(Wort)라고 하는 단위의 구성원이 되고, 이 단어들의 결합으로 문장(Satz)이 형성되는데 이러한 단어들의 결합을 기술하는 이론분야를 우리는 통사론(Syntax)이라고 말한다. 따라서 언어학의 이론분야는 다음과 같은 문법적인 단위를 구성원으로 하는 세부 분야의 위계(Hierarchie)적인 관계로 구성되어 있다.
 형태소는 문장이나 단어에 직접 나타나는 것이 아니라 문장이나 단어를 분절Segmentierung)하여 의미를 지니는 최소단위의 형태를 얻고 이 형태들을 분류(Klassifikation)하여 얻게 된다. 따라서 형태소란 의미를 지니는 최소단위이고 이러한 형태소를 찾기 위한 분석절차가 필요하게 된다.


100%|██████████████████████████████████████████████████████████████████████████| 115440/115440 [50:11<00:00, 38.33it/s]


주어진 제목과 요약문에 대한 정보를 토대로, 요약되기 전 문장을 유추해서 생성해주세요.

제목: 후진타오 시대의 중국정치
요약문: 공산당은 16차 당대회에서 선진세력의 입당을 공식 허용했다. 이는 “당 건설 강화에 대한 통치”라는 당 방침을 철회하고 사영기업가 계층의 합법적 지위를 인정했지만 이전에도 사영기업가가 자신의 정치적 지위를 높이려 했기 때문에 새롭고 획기적인 조치는 아니었다.
둘째, 제 16차 당 대회에서 공산당은 삼개대표 이론에 입각하여 생산력발전과 사회진보에 공헌한 자영업자(개체호)나 사영기업가 같은 ‘선진세력’의 입당을 공식 허용했다. 이것은 사영기업가의 입당을 불허한 13년 전의 당 방침, 즉 “당 건설 강화에 대한 통치”(1989)를 철회하고, 경제적 차원뿐만 아니라 정치적 차원에서도 사영기업가 계층에게 합법적 지위를 인정한 것이다. 물론 이 조치도 언론에 보도되었던 것처럼 새롭고 획기적인 것은 아니었다. 왜냐하면 이 조치 이전에도 1990년대에 들어 각 지방에서는 사영기업가가 하나의 사회계층으로 형성되면서 입당 외에 지방의회(지방인민대표대회/지방인대) 대표가 되거나 공익사업에 출연하는 등 여러 가지 방법을 통해 자신의 정치적 지위를 높이려는 노력을 꾸준히 해왔기 때문이다. 그래서 사영경제가 발달한 일부지방에서는 현급 지방의회 의원 중에서 20-40% 정도가 사영기업가 출신인 경우도 있었다. 따라서 이 조치는 이미 현실적으로 존재하는 사실을 사후 승인한 것이다.


100%|████████████████████████████████████████████████████████████████████████████| 13891/13891 [02:33<00:00, 90.22it/s]


주어진 문장 뒤에 자연스럽게 이어질 문장을 생성해주세요.

문장: 북한 연극에 대한 나의 탐구는 해방공간에 북으로 사라져 간 수많은 연극인들의 행적을 찾아보고자 하는 단순한 호기심에서 시작되었다.  해방공간에서 활동하던 연극인의 대다수가 납․월북의 과정을 거쳐 북한 연극계에 자리를 잡았기 때문이다.  그 안에는 극작가 송영, 함세덕, 박영호, 조영출, 연출가 이서향, 안영일, 신고송, 무대미술가 김일영, 강호, 배우 황철, 김선영, 문예봉, 만담가 신불출 등 기라성 같은 멤버들이 포함되어 있었다.  그 숫자로만 본다면 일제강점기 서울의 연극계가 통으로 평양으로 옮겨간 셈이었다.  그렇지만 이제 북한 연극에서 더 이상 그들의 존재를 확인하기 어려운 상황이다.  그들은 남에서도 북에서도 시계에서 영원히 사라져버린 ‘잃어버린 세대’ 그 자체이다.  그들의 흔적을 찾는 것은 차라리 고고학의 과제가 되었다.  그들이 역사의 저편으로 사라진 그 자리에 오늘의 북한 연극이 성채처럼 위용을 자랑하고 있다.  오늘날의 북한 연극은 모두가 주체사실주의에 입각하여 만들어지는 이른바 ‘<성황당>식 혁명연극’ 일색이다.  1978년 국립연극단의 <성황당> 공연의 성과를 본보기로 삼아 모든 연극이 ‘따라 배우기’를 하고 있기 때문이다.  북한의 연극과 희곡은 정점에서 내려 쏟는 단성적(單聲的) 문화회로 안에 갇혀 있다.  혁명연극 <성황당>(1978)의 본보기는 혁명가극 <피바다>(1971)이며, 그 근저에는 1960년대부터 시작된 김정일 주도의 문화예술혁명이 가로놓여 있다.  북한 연극의 창작과 향유, 그 모든 과정에서 김정일의 그림자에 맞닥뜨리지 않을 수 없다.  최근에 방문한 조선예술영화촬영소 에 있는 ‘문화성혁명사적관’(김정일관)에는 1960년대 중반부터 2000년대까지 40년 동안 김정일의 문화예술 부문 지도가 11,890건이며, 그 중 문화예술기관을 직접 방문하여 지도한 이른바 ‘현지지도’가 1,770건이라는 안내판이 있었다.
북한 연극이 김정일과 주체사상이라는 키워드를 떠

In [216]:
df.reset_index(drop=True, inplace=True)

In [217]:
df['input'] = ''

In [218]:
df

,input,instruction,output
0,,"주어진 제목과 요약문에 대한 정보를 토대로, 요약되기 전 문장을 유추해서 생성해주세...",지금의 국가가 직면하는 위기는 전통사회의 그것과 위기의 규모뿐만아니라 종류와 성질 ...
1,,"주어진 제목과 요약문에 대한 정보를 토대로, 요약되기 전 문장을 유추해서 생성해주세...",자연재해나 불량 수산식품 파동 등의 인위적 재난이 수산분야 피해로이어질 경우 일반분...
2,,"주어진 제목과 요약문에 대한 정보를 토대로, 요약되기 전 문장을 유추해서 생성해주세...",하지만 이와 반대로 MG 파동 이후 기간(2005.10～2011.12)의 경우에는 ...
3,,주어진 문장 뒤에 자연스럽게 이어질 문장을 생성해주세요.\n\n문장: 그러나 사회적...,정확한 관측은 조기 대응과 영향 분석을 비롯하여 장단기 대책수립에 많은 기여를 할 ...
4,,"주어진 제목과 요약문에 대한 정보를 토대로, 요약되기 전 문장을 유추해서 생성해주세...",네 번째는 공급감소로 국내 물량이 부족하게 되면 유통질서가 왜곡될 가능성이 많기 때...
...,...,...,...
159997,,"주어진 제목과 요약문에 대한 정보를 토대로, 요약되기 전 문장을 유추해서 생성해주세...",지도자로 등장한 2012년부터 2016년까지는 최고지도자로 추대되기 위한 준비기간이...
159998,,주어진 문장 뒤에 자연스럽게 이어질 문장을 생성해주세요.\n\n문장: 그러나 공공도...,"그러기 위해서 도서관 사명의 변화에 따른 서비스 범위의 변화가 필수적이며, 도서관 ..."
159999,,"주어진 제목과 요약문에 대한 정보를 토대로, 요약되기 전 문장을 유추해서 생성해주세...","작은도서관의 이용자가 이동에 제약이 있는 유아, 노인들처럼 도보만을 이용할 수 있는..."
160000,,주어진 문장 뒤에 자연스럽게 이어질 문장을 생성해주세요.\n\n문장: 공공도서관 사...,"또한, 정보해득력 강화를 위한 이용자교육과 국민독서활동을 지원하여 독서 및 문화활동..."


In [220]:
print(df.iloc[-1,1])
print(df.iloc[-1,2])

주어진 제목과 요약문에 대한 정보를 토대로, 요약되기 전 문장을 유추해서 생성해주세요.

제목: 2019 콘텐츠산업 통계조사 : 2018년 기준
요약문: 2018년 기준 지난 4년간 콘텐츠 산업 사업체 수는 감소하는 경향을 보였다. 콘텐츠 산업 내에서 음악산업이 차지하는 비중이 가장 컸으나 출판산업, 게임산업, 만화산업과 함께 지난 4년간 연평균 감소 추세를 보였다. 이와 반대로 콘텐츠 산업에서 차지하는 비중은 적지만 지식정보산업, 광고산업, 캐릭터 산업, 콘텐츠솔루션산업, 방송산업 및 애니메이션산업은 꾸준히 성장해왔다.
2018년 콘텐츠산업 사업체 수는 10만 5,310개로 전년대비 0.2% 감소했으며, 2014년부터 2018년까지 연평균 0.03% 감소했다. 산업별 사업체 수를 살펴본 결과, 음악산업 사업체 수가 3만 5,670개로 전체 콘텐츠산업 사업체 수의 33.9%를 차지해 가장 높은 비중을 보였다. 이는 전년대비 1.1% 감소, 연평균 0.6% 감소한 수치이다. 출판산업 사업체 수는 2만 4,995개로 23.7%의 비중을 차지했으며 전년대비 3.2% 감소, 2014년부터 2018년까지 연평균 0.7% 감소했다. 게임산업 사업체 수는 1만 3,357개로 12.7%의 비중을 차지했으며 전년대비 3.2% 증가, 연평균 1.9% 감소했다. 지식정보산업은 9,724개로 9.2%의 비중을 보였으며, 전년대비 6.3% 증가, 연평균 3.0% 증가했다. 광고산업은 7,256개로 6.9%의 비중을 차지했으며, 전년대비 0.3% 증가, 연평균 6.3% 증가했다. 만화산업은 6,628개로 전체 중 6.3%의 비중을 보였으며, 전년대비 7.6% 감소, 연평균 5.4% 감소했다. 캐릭터 산업은 2,534개로 2.4%의 비중을 차지했으며, 전년대비 12.1% 증가, 연평균 5.9% 증가했다. 콘텐츠솔루션산업은 2,120개로 2.0%의 비중을 보였으며, 전년대비 13.2% 증가, 연평균 7.5% 증가했다. 영화산업은 1,369개로 전체 1.3%의 비중을 차지했으며, 전년대비 

In [221]:
df.to_parquet(default_path+'Save/summary_com4.parquet', engine = 'pyarrow', index = False)

# 이후 나머지 Task에 대해서 데이터 추출

- 이후 생성된 finetuning 데이터셋과 corpus를 통해서 나머지 task에 대해서 데이터를 추출하는 작업을 진행합니다.

- `일방상식문장생성데이터.ipynb`에서는 위에서 저장한 일반상식 데이터를 가지고 fine-tuning 데이터를 생성하고, `논문자료요약.ipynb`에서는 위에서 생성한 논문자료 데이터를 가지고 fine-tuning 데이터를 생성하는 과정입니다.

- 둘중 순서 상관 없이 실행시키면 됩니다.
